NOTEBOOK CREADO Y EJECUTADO EN GOOGLE COLAB

**Importar librerias necesarias**

In [ ]:
# Uncomment the line for install ydata-profiling
#!pip install ydata-profiling

In [20]:
import pandas as pd
from ydata_profiling import ProfileReport
import matplotlib.pyplot as plt
import seaborn as sns

In [21]:
df = pd.read_csv('/content/the_oscar_award.csv')

In [22]:
print("Información del DataFrame (antes de limpiar):")
print(df.info())
print("\nValores faltantes por columna:")
print(df.isnull().sum())

Información del DataFrame (antes de limpiar):
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11110 entries, 0 to 11109
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   year_film       11110 non-null  int64 
 1   year_ceremony   11110 non-null  int64 
 2   ceremony        11110 non-null  int64 
 3   category        11110 non-null  object
 4   canon_category  11110 non-null  object
 5   name            11103 non-null  object
 6   film            10751 non-null  object
 7   winner          11110 non-null  bool  
dtypes: bool(1), int64(3), object(4)
memory usage: 618.6+ KB
None

Valores faltantes por columna:
year_film           0
year_ceremony       0
ceremony            0
category            0
canon_category      0
name                7
film              359
winner              0
dtype: int64


**Limpieza y transformación de datos**

In [23]:
df = df.dropna(subset=['name', 'film']).copy()
for col in df.select_dtypes(include=['object']).columns:
    df.loc[:, col] = df[col].str.strip()
df = df.drop_duplicates()

**Revisar que se hizo la limpieza de los datos**

In [24]:
print("\nInformación del DataFrame (después de limpiar):")
print(df.info())
print("\nValores faltantes por columna (después de limpiar):")
print(df.isnull().sum())
print(f"DataFrame limpio y transformado. Forma: {df.shape}")


Información del DataFrame (después de limpiar):
<class 'pandas.core.frame.DataFrame'>
Index: 10738 entries, 0 to 11105
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   year_film       10738 non-null  int64 
 1   year_ceremony   10738 non-null  int64 
 2   ceremony        10738 non-null  int64 
 3   category        10738 non-null  object
 4   canon_category  10738 non-null  object
 5   name            10738 non-null  object
 6   film            10738 non-null  object
 7   winner          10738 non-null  bool  
dtypes: bool(1), int64(3), object(4)
memory usage: 681.6+ KB
None

Valores faltantes por columna (después de limpiar):
year_film         0
year_ceremony     0
ceremony          0
category          0
canon_category    0
name              0
film              0
winner            0
dtype: int64
DataFrame limpio y transformado. Forma: (10738, 8)


**===== Análisis Exploratorio de Datos (EDA) =====**

In [25]:
print("Resumen estadístico de variables numéricas:")
print(df.describe())

Resumen estadístico de variables numéricas:
          year_film  year_ceremony      ceremony
count  10738.000000   10738.000000  10738.000000
mean    1977.135873    1978.135873     50.159434
std       27.640551      27.640551     27.600347
min     1927.000000    1928.000000      1.000000
25%     1953.000000    1954.000000     26.000000
50%     1976.000000    1977.000000     49.000000
75%     2002.000000    2003.000000     75.000000
max     2024.000000    2025.000000     97.000000


In [26]:
print("\nResumen estadístico de variables categóricas:")
print(df.describe(include=['object']))


Resumen estadístico de variables categóricas:
         category canon_category                 name            film
count       10738          10738                10738           10738
unique        112             53                 6697            5113
top     DIRECTING   BEST PICTURE  Metro-Goldwyn-Mayer  A Star Is Born
freq          476            611                   65              26


In [27]:
print("\nValores únicos por columna:")
for col in df.columns:
    print(f"{col}: {df[col].nunique()} únicos")


Valores únicos por columna:
year_film: 97 únicos
year_ceremony: 97 únicos
ceremony: 97 únicos
category: 112 únicos
canon_category: 53 únicos
name: 6697 únicos
film: 5113 únicos
winner: 2 únicos


In [28]:
print("\nFrecuencias de las principales columnas categóricas:")
cat_cols = df.select_dtypes(include=['object']).columns
for col in cat_cols:
    print(f"\nFrecuencias de '{col}':")
    print(df[col].value_counts().head(10))


Frecuencias de las principales columnas categóricas:

Frecuencias de 'category':
category
DIRECTING                       476
FILM EDITING                    455
ACTRESS IN A SUPPORTING ROLE    445
ACTOR IN A SUPPORTING ROLE      445
BEST PICTURE                    381
DOCUMENTARY (Short Subject)     378
DOCUMENTARY (Feature)           345
CINEMATOGRAPHY                  345
FOREIGN LANGUAGE FILM           315
ART DIRECTION                   309
Name: count, dtype: int64

Frecuencias de 'canon_category':
canon_category
BEST PICTURE                    611
MUSIC (Original Score)          524
ACTRESS IN A LEADING ROLE       484
WRITING (Adapted Screenplay)    482
ACTOR IN A LEADING ROLE         481
DIRECTING                       476
MUSIC (Original Song)           474
FILM EDITING                    455
ACTRESS IN A SUPPORTING ROLE    445
ACTOR IN A SUPPORTING ROLE      445
Name: count, dtype: int64

Frecuencias de 'name':
name
Metro-Goldwyn-Mayer    65
Walt Disney            60
John Wi

In [29]:
print("\nMedias, medianas y desviaciones estándar de variables numéricas:")
num_cols = df.select_dtypes(include=['number']).columns
for col in num_cols:
    print(f"{col}: media={df[col].mean()}, mediana={df[col].median()}, desviación estándar={df[col].std()}")


Medias, medianas y desviaciones estándar de variables numéricas:
year_film: media=1977.1358726019744, mediana=1976.0, desviación estándar=27.64055120575204
year_ceremony: media=1978.1358726019744, mediana=1977.0, desviación estándar=27.64055120575204
ceremony: media=50.159433786552434, mediana=49.0, desviación estándar=27.60034745094718


In [30]:
print("Generando reporte HTML de EDA con ydata-profiling...")
profile = ProfileReport(df, title="Reporte EDA - Oscars", explorative=True)
profile.to_file("oscar_eda_report.html")
print("Reporte HTML guardado como 'oscar_eda_report.html'.")

Generando reporte HTML de EDA con ydata-profiling...


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]


100%|██████████| 8/8 [00:00<00:00, 22.75it/s]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

Reporte HTML guardado como 'oscar_eda_report.html'.


**===== Generando Visualizaciones =====**

In [31]:
# 1. Histogram of awards by year
plt.figure(figsize=(12, 6))
sns.histplot(df['year_film'], bins=30, kde=True)
plt.title('Distribución de Premios Oscar por Año de Película')
plt.xlabel('Año de las Películas')
plt.ylabel('Número de Premios')
plt.grid(True)
plt.savefig('distribution_awards_year.png')
plt.close()
print("Histograma de premios por año guardado en 'distribution_awards_year.png'")

Histograma de premios por año guardado en 'distribution_awards_year.png'


In [32]:
# 2. Bar chart of top 10 most awarded films
plt.figure(figsize=(12, 8))
top_films = df['film'].value_counts().nlargest(10)
sns.barplot(x=top_films.values, y=top_films.index, hue=top_films.index, palette='viridis', legend=False)
plt.title('Top 10 Películas con Más Nominaciones/Premios Oscar')
plt.xlabel('Número de Nominaciones/Premios')
plt.ylabel('Película')
plt.tight_layout()
plt.savefig('most_awarded_movies.png')
plt.close()
print("Gráfico de barras de top 10 películas guardado en 'most_awarded_movies.png'")

Gráfico de barras de top 10 películas guardado en 'most_awarded_movies.png'


In [33]:
# 3. Bar chart of top 10 most frequent award categories
plt.figure(figsize=(12, 8))
top_categories = df['category'].value_counts().nlargest(10)
sns.barplot(x=top_categories.values, y=top_categories.index, hue=top_categories.index, palette='plasma', legend=False)
plt.title('Top 10 Categorías de Premios Oscar más Frecuentes')
plt.xlabel('Frecuencia')
plt.ylabel('Categoría')
plt.tight_layout()
plt.savefig('most_awarded_categories.png')
plt.close()
print("Gráfico de barras de top 10 categorías guardado en 'most_awarded_categories.png'")

Gráfico de barras de top 10 categorías guardado en 'most_awarded_categories.png'
